In [1]:
import networkx as nx
import custom_funcs as cf
import pandas as pd
import matplotlib.pyplot as plt
from Levenshtein import distance
from collections import defaultdict

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
G = nx.read_gpickle('20150902_all_ird Final Graph.pkl')
G = cf.clean_host_species_names(G)
G = cf.impute_reassortant_status(G)
G = cf.impute_weights(G)

In [3]:
G.nodes(data=True)[1286]

('A/chicken/Liaoning/0517/2013',
 {'collection_date': Timestamp('2013-05-17 00:00:00'),
  'country': 'China',
  'host_species': 'Chicken',
  'reassortant': False,
  'state': 'Liaoning',
  'subtype': 'H9N2'})

In [7]:
# Get all of the host species
host_species = list(set([d['host_species'] for n, d in G.nodes(data=True)]))
pd.Series(host_species).to_csv('host_species.csv')

In [42]:
cois = pd.read_csv('hosts_with_classification-NH_EM_mods.csv', index_col='Index').dropna(subset=['Sequence'])
cois_species = dict(zip(cois['Species'], cois['Sequence']))

In [43]:
for n, d in G.nodes(data=True):
    if d['host_species'] in cois_species.keys():
        G.node[n]['coi'] = cois_species[d['host_species']].replace('\r', '')
    else:
        G.node[n]['coi'] = None

In [54]:
# What is the distribution of COI distances across each edge?
ds_clonal = [] #distances under clonal descent
ws_clonal = [] #matched weights under clonal
ds_reassortant = [] #distances under reassortant descent
ws_reassortant = [] #matched weights under reassortant

for sc, sk, d in G.edges(data=True):
    if G.node[sc]['coi'] != None and G.node[sk]['coi'] != None:
        dist = distance(G.node[sc]['coi'], G.node[sk]['coi'])
        dist = dist
        
        if d['edge_type'] == 'reassortant':
            ds_reassortant.append(dist)
            ws_reassortant.append(d['weight'])
        else:
            ds_clonal.append(dist)
            ws_clonal.append(d['weight'])

In [58]:
weights_clonal = defaultdict(float)
for d, w in zip(ds_clonal, ws_clonal):
    weights_clonal[d] += w

In [59]:
weights_clonal

defaultdict(float,
            {0: 13066.366666666598,
             8: 1.0,
             24: 3.0,
             32: 2.0,
             35: 3.0,
             39: 2.0,
             43: 76.0,
             44: 4.0,
             46: 16.5,
             49: 217.50000000000006,
             53: 5.0,
             57: 30.000000000000007,
             58: 4.0,
             59: 26.0,
             61: 10.0,
             62: 36.300000000000004,
             63: 1.0,
             64: 3.0,
             65: 4.0,
             66: 12.0,
             67: 34.0,
             68: 2.0,
             69: 113.00000000000003,
             72: 3.0,
             73: 1.0,
             76: 88.0,
             78: 1.0,
             79: 1.0,
             80: 3.0,
             81: 4.0,
             82: 3.0,
             83: 7.0,
             85: 2.0,
             86: 2.0,
             89: 142.0,
             90: 1.0,
             91: 6.0,
             93: 1.0,
             94: 9.0,
             95: 6.0,
             97: 1.